# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read the weather csv file created from the WeatherPy analysis into a DF
weather_df=pd.read_csv('../output_data/weather.csv')
weather_df


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,237379,Ndélé,0,CF,1612288596,21,8.4109,20.6473,82.56,1.39
1,4032243,Vaini,90,TO,1612288597,100,-21.2000,-175.2000,77.00,14.97
2,3372707,Ribeira Grande,40,PT,1612288600,62,38.5167,-28.7000,57.20,26.46
3,6170031,Tuktoyaktuk,20,CA,1612288601,68,69.4541,-133.0374,-29.20,5.75
4,5866583,Saint Paul Harbor,90,US,1612288565,86,57.7900,-152.4072,26.60,4.99
...,...,...,...,...,...,...,...,...,...,...
537,482986,Tim,90,RU,1612289239,86,51.6222,37.1244,24.80,6.71
538,103630,Najrān,0,SA,1612289240,32,17.4924,44.1277,68.00,6.91
539,1261309,Nipāni,0,IN,1612289240,34,16.4000,74.3833,71.20,5.23
540,3372783,Ponta Delgada,20,PT,1612289241,67,37.7333,-25.6667,57.20,24.16


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#set the latitude and longitudes as locations
locations=weather_df[['Lat','Lng']]
#set the humidity as the weight
weight=weather_df['Humidity']

#set colors for the gradient levels of the heat map
gradient = ["rgba(0, 255, 255, 0)",
            "rgba(0, 255, 255, 1)",
            "rgba(0, 191, 255, 1)",
            "rgba(0, 127, 255, 1)",
            "rgba(0, 63, 255, 1)",
            "rgba(0, 0, 255, 1)",
            "rgba(0, 0, 223, 1)",
            "rgba(0, 0, 191, 1)",
            "rgba(0, 0, 159, 1)",
            "rgba(0, 0, 127, 1)",
            "rgba(63, 0, 91, 1)",
            "rgba(127, 0, 63, 1)",
            "rgba(191, 0, 31, 1)",
            "rgba(255, 0, 0, 1)",]

#setup your figure dimensions
figure_layout = {'width': '300px','height': '300px',
                 'border': '1px solid black',
                 'padding': '1px','margin': '0 auto 0 auto'}
#create figure using the layout from above
fig = gmaps.figure(layout=figure_layout)

# create heat layer and formatting
heat = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=125, 
        point_radius=5, gradient=gradient,)

#add heat layer to map figure
fig.add_layer(heat)

fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
  - A max temperature lower than 75 degrees but higher than 60.
  - Wind speed less than 10 mph.
  - Humidity is less than 50%
* Drop any rows that don't contain all three conditions, or are null

In [4]:
#pull the cities from the weather DF that meet all three conditions 
#and create a new df with just those cities
weather_df2 = np.where((weather_df['Max Temp'] < 70) & (weather_df['Max Temp'] > 60) & (weather_df['Wind Speed'] < 10) & (weather_df.Humidity < 50))
ideal_weather_df = weather_df.loc[weather_df2]
ideal_weather_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
158,1260009,Pawai,0,IN,1612288767,31,24.2667,80.1667,61.11,5.59
177,347796,Sohag,0,EG,1612288791,32,26.5570,31.6948,68.00,6.91
234,293842,Kiryat Gat,0,IL,1612288849,42,31.6100,34.7642,68.00,3.65
318,362973,Abnūb,0,EG,1612288939,42,27.2696,31.1511,66.20,6.91
319,359792,Aswān,0,EG,1612288941,24,24.0934,32.9070,67.21,3.51
376,3985797,San Julián,82,MX,1612289065,32,21.0167,-102.1667,65.73,5.37
382,4726521,San Patricio,20,US,1612289074,37,28.0170,-97.5169,68.00,8.05
448,2446796,Bilma,0,NE,1612289143,22,18.6853,12.9164,69.57,8.55
460,1254709,Tezu,5,IN,1612289158,42,27.9167,96.1667,61.09,3.83
501,286987,Nizwá,0,OM,1612289203,30,22.9333,57.5333,60.57,2.48


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#make a copy of the ideal_weather_df and add column for Hotel Name
hotel_df=ideal_weather_df.copy()
hotel_df['Hotel Name']=''
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
158,1260009,Pawai,0,IN,1612288767,31,24.2667,80.1667,61.11,5.59,
177,347796,Sohag,0,EG,1612288791,32,26.5570,31.6948,68.00,6.91,
234,293842,Kiryat Gat,0,IL,1612288849,42,31.6100,34.7642,68.00,3.65,
318,362973,Abnūb,0,EG,1612288939,42,27.2696,31.1511,66.20,6.91,
319,359792,Aswān,0,EG,1612288941,24,24.0934,32.9070,67.21,3.51,
376,3985797,San Julián,82,MX,1612289065,32,21.0167,-102.1667,65.73,5.37,
382,4726521,San Patricio,20,US,1612289074,37,28.0170,-97.5169,68.00,8.05,
448,2446796,Bilma,0,NE,1612289143,22,18.6853,12.9164,69.57,8.55,
460,1254709,Tezu,5,IN,1612289158,42,27.9167,96.1667,61.09,3.83,
501,286987,Nizwá,0,OM,1612289203,30,22.9333,57.5333,60.57,2.48,


In [6]:
#create the base url for nearby searches in gmap
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#define parameters to use in our api call
params = {'type': 'lodging', 'radius':5000,'key':g_key,}
#loop thru df to find if there is hotel/lodging within radius, and 
#pass hotel name into the Hotel Name column or note that one is not found
for i, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat}, {lng}'
    response = requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[i,'Hotel Name'] = response['results'][0]['name']
    except:
        hotel_df.loc[i,'Hotel Name'] = 'Hotel not found within radius'
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
158,1260009,Pawai,0,IN,1612288767,31,24.2667,80.1667,61.11,5.59,parth Palace Pawai
177,347796,Sohag,0,EG,1612288791,32,26.5570,31.6948,68.00,6.91,Civil Registry - Sohag 1
234,293842,Kiryat Gat,0,IL,1612288849,42,31.6100,34.7642,68.00,3.65,OlusHome
318,362973,Abnūb,0,EG,1612288939,42,27.2696,31.1511,66.20,6.91,Romany Adly Gad
319,359792,Aswān,0,EG,1612288941,24,24.0934,32.9070,67.21,3.51,Nuba Nile
376,3985797,San Julián,82,MX,1612289065,32,21.0167,-102.1667,65.73,5.37,La Nueva España
382,4726521,San Patricio,20,US,1612289074,37,28.0170,-97.5169,68.00,8.05,"Motel 6 Sinton, TX"
448,2446796,Bilma,0,NE,1612289143,22,18.6853,12.9164,69.57,8.55,Expeditions Ténére voyages
460,1254709,Tezu,5,IN,1612289158,42,27.9167,96.1667,61.09,3.83,Circuit House
501,286987,Nizwá,0,OM,1612289203,30,22.9333,57.5333,60.57,2.48,Orient Apartments


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…